In [1]:
import do_mpc
from casadi import vertcat, cross, SX, MX, norm_fro

/home/led/miniconda3/lib/python3.11/site-packages/do_mpc/sysid/__init__.py:15: UserWarning: The ONNX feature is not available. Please install the full version of do-mpc to access this feature.
  warnings.warn('The ONNX feature is not available. Please install the full version of do-mpc to access this feature.')
/home/led/miniconda3/lib/python3.11/site-packages/do_mpc/opcua/__init__.py:14: UserWarning: The opcua feature is not available. Please install the full version of do-mpc to access this feature.
  warnings.warn('The opcua feature is not available. Please install the full version of do-mpc to access this feature.')


In [2]:
import numpy as np

In [3]:
C_t = 0.112
C_f = 0.98
m = 1

model_type = 'continuous'
model = do_mpc.model.Model(model_type, symvar_type='MX')
rps = model.set_variable(var_type='_u', var_name='rps', shape=(4, 1))

vel = model.set_variable(var_type='_x', var_name='vel', shape=(3, 1))
# ang_acc = model.set_variable(var_type='_x', var_name='ang_acc', shape=(3, 1))

targ_vec = model.set_variable(var_type='_x', var_name='targ', shape=(3, 1))


ang_vel = model.set_variable(var_type='_x', var_name='ang_vel', shape=(3, 1))
moments = model.set_variable(var_type='_x', var_name='moments', shape=(3, 1))
# forces = model.set_variable(var_type='_x', var_name='forces', shape=(3, 1))

acc_eq = vertcat(
    0, 0, 
    # C_f*(rps[0]**2+rps[1]**2+rps[2]**2+rps[3]**2)/m 
    C_f*MX.ones(1, 4)@(rps**2)/m
)
model.set_rhs("vel", acc_eq)

x_dir = MX([-1, 1, -1, 1])
y_dir = MX([-1, -1, 1, 1])
z_dir = MX([-1, 1, 1, -1])

I = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

I_inv = np.linalg.inv(I)

moments_eq = vertcat(
    x_dir.T@(C_f*rps**2), 
    y_dir.T@(C_f*rps**2), 
    z_dir.T@(C_t*rps**2), 
)
model.set_rhs("moments", moments_eq)

ang_acc_eq = I_inv@(moments - cross(ang_vel, I@ang_vel))
model.set_rhs("ang_vel", ang_acc_eq)

model.setup()

In [20]:
desired_vector = np.array([0, 0, 1])

mpc = do_mpc.controller.MPC(model)

setup_mpc = {
    'n_horizon': 20,
    't_step': 0.1,
    'n_robust': 1,
    'store_full_solution': True,
}
mpc.set_param(**setup_mpc)

vel_dir = vel/norm_fro(vel)
# vel_dir = vel_/np.linalg.norm(vel_)
term = vel_dir.T@desired_vector

mpc.set_objective(mterm=term, lterm=term)
mpc.set_rterm(
    rps = np.array([0.001, 0.01, 0.01, 0.01])
)
mpc.setup()



In [15]:
estimator = do_mpc.estimator.StateFeedback(model)
simulator = do_mpc.simulator.Simulator(model)
simulator.set_param(t_step = 0.1)
simulator.setup()

In [16]:
x0 = np.zeros((9,1))

In [17]:
simulator.x0 = x0
mpc.x0 = x0

In [18]:
mpc.set_initial_guess()

AssertionError: MPC was not setup yet. Please call MPC.setup().